In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.1

In [2]:
import os, yaml, json, time
import numpy as np
import pandas as pd
import torch
import optuna
from ultralytics import YOLO
from datetime import datetime

# Define constants
SEED = 42
DATA_ROOT = "/kaggle/input/yolo-data/yolo_dataset"
WORK_DIR = "/kaggle/working/yolo_flagella"
N_TRIALS = 20  # Number of hyperparameter trials to run
os.makedirs(WORK_DIR, exist_ok=True)

# Create data YAML
data_yaml = {
    "path": DATA_ROOT,
    "train": "images/train",
    "val": "images/val" if os.path.exists(f"{DATA_ROOT}/images/val") else "images/train",
    "nc": 1,
    "names": ["motor"]
}
DATA_YAML_PATH = f"{WORK_DIR}/flagella.yaml"
with open(DATA_YAML_PATH, "w") as f:
    yaml.dump(data_yaml, f)
print("Created data yaml:", DATA_YAML_PATH)

# Set seeds for reproducibility
np.random.seed(SEED)
torch.manual_seed(SEED)

# Define the objective function for Optuna
def objective(trial):
    # Generate unique trial name
    trial_name = f"trial_{trial.number}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    # Sample hyperparameters
    hyperparams = {
        # Learning rates - log scale for better sampling
        'lr0': trial.suggest_float('lr0', 1e-5, 1e-3, log=True),
        'lrf': trial.suggest_float('lrf', 0.001, 0.01, log=True),
        
        # Augmentation parameters
        'mosaic': trial.suggest_float('mosaic', 0.5, 0.9),
        'mixup': trial.suggest_float('mixup', 0.0, 0.2),
        'copy_paste': trial.suggest_float('copy_paste', 0.0, 0.1),
        'hsv_h': trial.suggest_float('hsv_h', 0.0, 0.05),
        'hsv_s': trial.suggest_float('hsv_s', 0.0, 0.3),
        'hsv_v': trial.suggest_float('hsv_v', 0.0, 0.3),
        'degrees': trial.suggest_int('degrees', 0, 180),
        'fliplr': trial.suggest_float('fliplr', 0.0, 0.5),
        'flipud': trial.suggest_float('flipud', 0.0, 0.5),
        
        # Loss weights
        'box': trial.suggest_float('box', 0.02, 0.2),
        'cls': trial.suggest_float('cls', 0.2, 0.8),
        
        # Training parameters
        'batch': trial.suggest_categorical('batch', [8, 16, 32]),
        'weight_decay': trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True),
    }
    
    # Log the parameters we're trying
    print(f"\n=== Trial {trial.number} ===")
    for k, v in hyperparams.items():
        print(f"{k}: {v}")
    
    # Initialize model
    model = YOLO('yolov8s.pt')
    
    try:
        # Train with these hyperparameters
        results = model.train(
            data=DATA_YAML_PATH,
            imgsz=640,
            epochs=50,  # Reduced for faster trials
            project=WORK_DIR,
            name=trial_name,
            exist_ok=True,
            seed=SEED,
            single_cls=True,
            patience=15,  # Early stopping to speed up trials
            **hyperparams
        )
        
        # Extract primary metric
        map_val = results.results_dict.get('metrics/mAP50-95(B)', 0)
        
        # Store all metrics in trial user attributes
        trial.set_user_attr('mAP50', results.results_dict.get('metrics/mAP50(B)', 0))
        trial.set_user_attr('precision', results.results_dict.get('metrics/precision(B)', 0))
        trial.set_user_attr('recall', results.results_dict.get('metrics/recall(B)', 0))
        
        return map_val
        
    except Exception as e:
        print(f"Error in trial {trial.number}: {str(e)}")
        return 0  # Return worst possible score on error
    
# Create an Optuna study
study = optuna.create_study(direction='maximize', 
                           study_name='flagellar_motor_detection')

# Run the optimization
study.optimize(objective, n_trials=N_TRIALS)

# Get best trial and parameters
best_trial = study.best_trial
best_params = best_trial.params
best_map = best_trial.value

print("\n" + "="*50)
print(f"Best trial: #{best_trial.number}")
print(f"Best mAP50-95: {best_map:.4f}")
print("Best hyperparameters:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

# Save all trials results to CSV
trials_df = study.trials_dataframe()
trials_df.to_csv(f"{WORK_DIR}/optuna_trials_results.csv", index=False)

# Save best parameters to JSON
with open(f"{WORK_DIR}/best_params.json", 'w') as f:
    json.dump(best_params, f, indent=4)

# Visualize optimization history
try:
    from optuna.visualization import plot_optimization_history, plot_param_importances
    import matplotlib.pyplot as plt
    
    # Plot optimization history
    fig = plot_optimization_history(study)
    fig.write_image(f"{WORK_DIR}/optimization_history.png")
    
    # Plot parameter importances
    fig = plot_param_importances(study)
    fig.write_image(f"{WORK_DIR}/param_importances.png")
    
    print(f"Plots saved to {WORK_DIR}")
except:
    print("Visualization libraries not available. Install plotly for visualization.")

print("\nTraining complete with best parameters:")
print(best_params)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


[I 2025-05-16 09:34:13,423] A new study created in memory with name: flagellar_motor_detection


Created data yaml: /kaggle/working/yolo_flagella/flagella.yaml

=== Trial 0 ===
lr0: 0.00029540897720910675
lrf: 0.007947800117728282
mosaic: 0.7987560065606213
mixup: 0.17216840207742765
copy_paste: 0.022768861093074577
hsv_h: 0.007012038053615444
hsv_s: 0.1783301827420825
hsv_v: 0.2091159807174562
degrees: 124
fliplr: 0.26370724070217955
flipud: 0.4420908058986989
box: 0.12262251967986024
cls: 0.5003127233391899
batch: 16
weight_decay: 3.061447323282775e-05


100%|██████████| 21.5M/21.5M [00:00<00:00, 136MB/s]


Ultralytics 8.3.135 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.12262251967986024, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5003127233391899, conf=None, copy_paste=0.022768861093074577, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_flagella/flagella.yaml, degrees=124, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.26370724070217955, flipud=0.4420908058986989, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.007012038053615444, hsv_s=0.1783301827420825, hsv_v=0.2091159807174562, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00029540897720910675, lrf=0.007947800117728282, mask_ratio=4, max_det=300, mixup=0.17216840207742765, mode=train, model=yolov8s.pt, momen

...

Error in trial 17: Dataset '/kaggle/working/yolo_flagella/flagella.yaml' error ❌ Dataset '/kaggle/working/yolo_flagella/flagella.yaml' images not found, missing path '/kaggle/input/yolo-data/yolo_dataset/images/train'
Note dataset download directory is '/kaggle/working/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'

=== Trial 18 ===
lr0: 0.00019319010470530612
lrf: 0.002012173657364943
mosaic: 0.5038958688493242
mixup: 0.17896157174537217
copy_paste: 0.021239324705904894
hsv_h: 0.0061087146292231175
hsv_s: 0.17416433098810752
hsv_v: 0.1949537206350005
degrees: 95
fliplr: 0.26775480021110354
flipud: 0.30968050367860456
box: 0.06435398294808635
cls: 0.5996973551660331
batch: 16
weight_decay: 0.0009753777070791163
Ultralytics 8.3.135 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.06435398294808635, cache=False, cfg=None, c

[I 2025-05-16 09:34:17,412] Trial 18 finished with value: 0.0 and parameters: {'lr0': 0.00019319010470530612, 'lrf': 0.002012173657364943, 'mosaic': 0.5038958688493242, 'mixup': 0.17896157174537217, 'copy_paste': 0.021239324705904894, 'hsv_h': 0.0061087146292231175, 'hsv_s': 0.17416433098810752, 'hsv_v': 0.1949537206350005, 'degrees': 95, 'fliplr': 0.26775480021110354, 'flipud': 0.30968050367860456, 'box': 0.06435398294808635, 'cls': 0.5996973551660331, 'batch': 16, 'weight_decay': 0.0009753777070791163}. Best is trial 0 with value: 0.0.


Error in trial 18: Dataset '/kaggle/working/yolo_flagella/flagella.yaml' error ❌ Dataset '/kaggle/working/yolo_flagella/flagella.yaml' images not found, missing path '/kaggle/input/yolo-data/yolo_dataset/images/train'
Note dataset download directory is '/kaggle/working/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'

=== Trial 19 ===
lr0: 0.0003647052053022239
lrf: 0.007344340712073072
mosaic: 0.8446797164166171
mixup: 0.12188358365581164
copy_paste: 0.04924607962134962
hsv_h: 0.022839880378920915
hsv_s: 0.2901988077551627
hsv_v: 0.07015301786102993
degrees: 134
fliplr: 0.1716804614546295
flipud: 0.45664324651542904
box: 0.104894651309579
cls: 0.3599797531153722
batch: 8
weight_decay: 2.412450428250178e-05
Ultralytics 8.3.135 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=0.104894651309579, cache=False, cfg=None, classes=No

[I 2025-05-16 09:34:17,595] Trial 19 finished with value: 0.0 and parameters: {'lr0': 0.0003647052053022239, 'lrf': 0.007344340712073072, 'mosaic': 0.8446797164166171, 'mixup': 0.12188358365581164, 'copy_paste': 0.04924607962134962, 'hsv_h': 0.022839880378920915, 'hsv_s': 0.2901988077551627, 'hsv_v': 0.07015301786102993, 'degrees': 134, 'fliplr': 0.1716804614546295, 'flipud': 0.45664324651542904, 'box': 0.104894651309579, 'cls': 0.3599797531153722, 'batch': 8, 'weight_decay': 2.412450428250178e-05}. Best is trial 0 with value: 0.0.


Error in trial 19: Dataset '/kaggle/working/yolo_flagella/flagella.yaml' error ❌ Dataset '/kaggle/working/yolo_flagella/flagella.yaml' images not found, missing path '/kaggle/input/yolo-data/yolo_dataset/images/train'
Note dataset download directory is '/kaggle/working/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'

Best trial: #0
Best mAP50-95: 0.0000
Best hyperparameters:
  lr0: 0.00029540897720910675
  lrf: 0.007947800117728282
  mosaic: 0.7987560065606213
  mixup: 0.17216840207742765
  copy_paste: 0.022768861093074577
  hsv_h: 0.007012038053615444
  hsv_s: 0.1783301827420825
  hsv_v: 0.2091159807174562
  degrees: 124
  fliplr: 0.26370724070217955
  flipud: 0.4420908058986989
  box: 0.12262251967986024
  cls: 0.5003127233391899
  batch: 16
  weight_decay: 3.061447323282775e-05
Visualization libraries not available. Install plotly for visualization.

Training complete with best parameters:
{'lr0': 0.00029540897720910675, 'lrf': 0.007947800117728282, 'mosa